In [1]:
%matplotlib inline


# Numerical libraries
import numpy as np   

# Import Linear Regression machine learning library
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score

# to handle data in form of rows and columns 
import pandas as pd    

# importing ploting libraries
import matplotlib.pyplot as plt   

#importing seaborn for statistical plots
import seaborn as sns

In [40]:
mpg_df = pd.read_csv("car-mpg.csv")  
mpg_df = mpg_df.drop('car_name', axis=1)
mpg_df['origin'] = mpg_df['origin'].replace({1: 'america', 2: 'europe', 3: 'asia'})
# Dummy variable - drop_first=True to ensure one less dummy variable is defined
mpg_df = pd.get_dummies(mpg_df, columns=['origin'],drop_first=True)
mpg_df = mpg_df.replace('?', np.nan)
mpg_df = mpg_df.apply(lambda x: x.fillna(x.median()),axis=0)
mpg_df['hp'] = mpg_df['hp'].astype('float64')

In [58]:
mpg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
mpg              398 non-null float64
cyl              398 non-null int64
disp             398 non-null float64
hp               398 non-null float64
wt               398 non-null int64
acc              398 non-null float64
yr               398 non-null int64
car_type         398 non-null int64
origin_asia      398 non-null int64
origin_europe    398 non-null int64
dtypes: float64(4), int64(6)
memory usage: 31.2 KB


# separate independent and dependent variables

In [42]:
# Copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = mpg_df.drop('mpg', axis=1)

# Copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = mpg_df[['mpg']]


In [43]:
#from sklearn import preprocessing

# scale all the columns of the mpg_df. This will produce a numpy array
#X_scaled = preprocessing.scale(X)
#X_scaled = pd.DataFrame(X_scaled, columns=X.columns) 

In [44]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)  
X_scaled = pd.DataFrame(X_scaled, columns=X.columns) 

In [45]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.30, random_state=1)

# fit a simple linear model

In [46]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

print("Intercept: ", regression_model.intercept_)
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

Intercept:  [23.66510774]
The coefficient for cyl is 2.5059518049385003
The coefficient for disp is 2.5357082860560567
The coefficient for hp is -1.788933573632524
The coefficient for wt is -5.551819873098731
The coefficient for acc is 0.11485734803440942
The coefficient for yr is 2.9318465482116087
The coefficient for car_type is 2.977869737601941
The coefficient for origin_asia is 0.8282270142957167
The coefficient for origin_europe is 0.836278138394881


In [47]:
print(regression_model.score(X_train, y_train))
print(regression_model.score(X_test, y_test))


0.8343770256960538
0.8513421387780066


# Create a regularized RIDGE model and note the coefficients

In [48]:
ridge = Ridge(alpha=1)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))



Ridge model: [[ 2.38873168  2.24584874 -1.77595406 -5.30121587  0.0716231   2.90348182
   2.87200551  0.8120093   0.81048147]]


In [49]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.8342199644938272
0.852973535261167


# Create a regularized LASSO model and note the coefficients

In [50]:
lasso = Lasso(alpha=1)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))

# Observe, many of the coefficients have become 0 indicating drop of those dimensions from the model

Lasso model: [-0.         -0.         -0.05656463 -4.05009448  0.          2.03034296
  0.95988039  0.          0.        ]


In [51]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

0.7821044353977104
0.8251153919895945


## Let us compare their scores

In [52]:
print(regression_model.score(X_train, y_train))
print(regression_model.score(X_test, y_test))


0.8343770256960538
0.8513421387780066


In [53]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.8342199644938272
0.852973535261167


In [54]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

0.7821044353977104
0.8251153919895945


# Let us generate polynomial models reflecting the non-linear interaction between some dimensions

In [55]:
from sklearn.preprocessing import PolynomialFeatures

In [56]:
poly = PolynomialFeatures(degree = 2, include_bias=False)


In [57]:
X_poly = poly.fit_transform(X_scaled )
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.30, random_state=1)
X_train.shape

(278, 54)

In [ ]:
#Formating of output can be done by getting columns using following command
#poly.get_feature_names(X_scaled.columns)

# Fit a simple non regularized linear model on poly features-

In [23]:
regression_model.fit(X_train, y_train)
print(regression_model.coef_[0])


[-2.15250620e+12 -5.30456543e+00 -2.75573730e+00 -2.00439453e+00
 -1.58306885e+00  3.07458496e+00 -2.72428145e+12  5.34086113e+11
  3.44884114e+11  2.25551631e+12 -3.40027618e+00 -1.39752197e+00
  4.24926758e-01  1.97970581e+00 -1.71374512e+00 -3.40148148e+12
  6.42530871e+12  5.12204617e+12  1.98443604e+00  8.92944336e-02
 -8.43322754e-01 -5.07141113e-01  3.82293701e+00 -1.96130371e+00
 -6.47592163e+00 -1.01669312e+00 -4.46380615e-01  1.05938721e+00
 -1.51458740e+00 -1.14929199e+00 -1.40612793e+00  9.98504639e-01
  1.50529480e+00 -2.19665527e-01  4.12597656e-01 -2.81311035e-01
 -3.28540039e+00  2.19372559e+00  5.66253662e-02 -2.55096436e-01
  4.55627441e-01  8.48297119e-01 -6.53076172e-03  1.03564453e+00
  1.07699585e+00  2.24853516e-01  7.11853027e-01  1.08731079e+00
  3.29155551e+11  3.77523367e+12  3.00949294e+12  8.07446285e+11
 -4.18360095e+11  5.87228269e+11]


In [24]:
print(regression_model.score(X_train, y_train))
print(regression_model.score(X_test, y_test))

0.9227971063626983
0.8482572586962259


## Ridge - modify ALPHA values to get better score

In [27]:
ridge = Ridge(alpha=1)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))

Ridge model: [[ 2.63865541e+00 -2.37408486e+00 -2.65212967e+00 -3.12107114e+00
  -1.12746233e+00  3.07077235e+00  1.52909161e+00  2.02604877e-01
  -2.87629850e-01 -1.37945713e+00 -1.09966208e+00 -1.01078797e+00
   3.32260057e-01  1.56812916e+00 -9.84590552e-01  1.30667566e-02
   2.36932639e+00 -1.39028192e+00  4.81491658e-01  2.39331914e-01
   9.28459487e-02 -2.52840506e-01  2.70741772e+00 -1.61539650e+00
  -9.84221502e-01 -4.98508396e-01 -1.74610449e-01  2.22020751e-01
  -9.95456295e-01 -1.05636540e+00 -1.36102812e+00  6.38279930e-01
   1.17200558e+00 -9.22402736e-02 -1.58827979e-02 -1.17973218e-01
  -2.39724426e+00  1.94066565e-01 -1.01230473e-03 -1.36767707e-01
   4.02066040e-01  4.16867222e-01  5.62555026e-01  1.02873950e+00
   1.09019774e+00  2.10712626e-01  6.45560484e-01  1.00792770e+00
  -1.84749264e-01  9.68725309e-01  4.50008326e-01  3.06303705e-01
   4.95400677e-02 -4.89742415e-01]]


In [28]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.920098330412219
0.8572159964789556


## LASSO - modify ALPHA values to get better score

In [29]:
lasso = Lasso(alpha=1)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))

Lasso model: [-0.         -0.         -0.06363396 -4.05182111  0.          2.0347072
  0.9255677   0.          0.         -0.          0.          0.
  0.         -0.         -0.          0.         -0.         -0.
  0.          0.          0.         -0.         -0.         -0.
 -0.         -0.          0.          0.         -0.         -0.
 -0.         -0.         -0.          0.         -0.         -0.
 -0.         -0.         -0.          0.          0.          0.
  0.          0.12469132  0.          0.          0.          0.
 -0.          0.          0.          0.0418751  -0.          0.        ]


In [31]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

0.787130614218031
0.828521489012022
